# Check implementation of Wajima-Hartmann model

In [1]:
import numpy as np
import plotly.graph_objects as go

from model import define_wajima_model

## Define Wajima model

In [2]:
mechanistic_model, parameters_df = define_wajima_model()

## Reproduce Figure 2 in Wajima et al (2009)
Time course of coagulation factors and integral of fibrin during PT test
and aPPT test.

For details on the tests, see documentation.

### 1. Prothrombin test

In [3]:
# Prepare model for test
outputs = [
    'central.coagulation_factor_ii_concentration',
    'central.activated_coagulation_factor_ii_concentration',
    'central.coagulation_factor_x_concentration',
    'central.activated_coagulation_factor_x_concentration',
    'central.fibrinogen_concentration',
    'central.fibrin_concentration']
mechanistic_model.set_outputs(outputs)
parameters = np.array([
    parameters_df[parameters_df.Parameter == p].Value.values[0]
    for p in mechanistic_model.parameters()
])

# Dilute the plasma sample
n_states = mechanistic_model._n_states
parameters[:n_states] /= 3

# Set thrombomodulin concentration to zero
parameter_names = np.array(mechanistic_model.parameters())
index = np.where(parameter_names == 'central.thrombomodulin_amount')[0]
parameters[index] = 0

# Set endogenous production rates to zero
index = np.where(parameter_names == 'myokit.input_rate_vk')[0]
parameters[index] = 0
index = np.where(parameter_names == 'myokit.production_rate_fibrinogen')[0]
parameters[index] = 0
index = np.where(
    parameter_names == 'myokit.production_rate_plasminogen')[0]
parameters[index] = 0
index = np.where(
    parameter_names == 'myokit.production_rate_prekrallikrein')[0]
parameters[index] = 0
index = np.where(parameter_names == 'myokit.production_rate_tfpi')[0]
parameters[index] = 0
index = np.where(parameter_names == 'myokit.production_rate_tmod')[0]
parameters[index] = 0
index = np.where(parameter_names == 'myokit.production_rate_v')[0]
parameters[index] = 0
index = np.where(parameter_names == 'myokit.production_rate_viii')[0]
parameters[index] = 0
index = np.where(parameter_names == 'myokit.production_rate_xi')[0]
parameters[index] = 0
index = np.where(parameter_names == 'myokit.production_rate_xii')[0]
parameters[index] = 0
index = np.where(parameter_names == 'myokit.production_rate_xiii')[0]
parameters[index] = 0

# Make sure that only compounds in central compartment are considered
index = np.where(parameter_names == 'dose.drug_amount')[0]
parameters[index] = 0
index = np.where(
    parameter_names == 'peripheral_vitamin_k.vitamin_k_peripheral_amount'
)[0]
parameters[index] = 0
index = np.where(
    parameter_names == 'myokit.transition_rate_vk_central_to_peripheral'
)[0]
parameters[index] = 0
index = np.where(
    parameter_names == 'myokit.transition_rate_vk_peripheral_to_central'
)[0]
parameters[index] = 0

# Change time units to seconds (for numerical stability)
indices = []
for idp, name in enumerate(parameter_names):
    if 'rate' in name:
        indices.append(idp)
parameters[indices] /= 60 * 60

# Add 300 nM tissue factor
index = np.where(parameter_names == 'central.size')[0]
volume = parameters[index]
index = np.where(parameter_names == 'central.tissue_factor_amount')[0]
parameters[index] = 300 * volume

# Ensure excess of coagulation factor VII (set to 10 nM)
index = np.where(
    parameter_names == 'central.caogaulation_factor_vii_amount')[0]
parameters[index] = 10 * volume

# Integrate fibrin concentration curve
times = np.linspace(start=0, stop=90, num=1000)
delta_time = (times[1] - times[0])
simulation = mechanistic_model.simulate(parameters, times)
fibrin_auc = np.cumsum(simulation[-1] * delta_time)

Visualise results

In [4]:
# Figure 1: Change of coagulation factors
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=times,
    y=simulation[0] / simulation[0, 0],
    name='Prothrombin'
))
fig.add_trace(go.Scatter(
    x=times,
    y=simulation[1] / simulation[0, 0],
    name='Thrombin'
))
fig.add_trace(go.Scatter(
    x=times,
    y=simulation[2] / simulation[2, 0],
    name='Coag. factor X'
))
fig.add_trace(go.Scatter(
    x=times,
    y=simulation[3] / simulation[2, 0],
    name='Act. coag. factor X'
))
fig.add_trace(go.Scatter(
    x=times,
    y=simulation[4] / simulation[4, 0],
    name='Fibrinogen'
))
fig.add_trace(go.Scatter(
    x=times,
    y=simulation[5] / simulation[4, 0],
    name='Fibrin'
))
fig.update_layout(
    xaxis_title='Time in seconds',
    yaxis_title=r'% of initial inactivated concentration',
    yaxis_range=[0, 1]
)
fig.show()

# Figure 2: Fibrin integral
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=times,
    y=fibrin_auc,
))
fig.add_shape(type="line",
    x0=0, y0=1500, x1=11.75, y1=1500,
    line=dict(
        color="red",
        width=2,
        dash="dash"
    )
)
fig.add_shape(type="line",
    x0=11.75, y0=-1, x1=11.75, y1=1500,
    line=dict(
        color="red",
        width=2,
        dash="dash",
    )
)
fig.update_layout(
    xaxis_title='Time in days',
    yaxis_title='Integral of fibrin in nMs',
    yaxis_type='log',
    yaxis_range=[-1, 6]
)
fig.show()

### 2. aPPT test (not relevant for this study, but useful as a implementation check)

In [5]:
# Prepare model for test
outputs = [
    'central.coagulation_factor_ii_concentration',
    'central.activated_coagulation_factor_ii_concentration',
    'central.coagulation_factor_x_concentration',
    'central.activated_coagulation_factor_x_concentration',
    'central.fibrinogen_concentration',
    'central.fibrin_concentration']
mechanistic_model.set_outputs(outputs)
parameters = np.array([
    parameters_df[parameters_df.Parameter == p].Value.values[0]
    for p in mechanistic_model.parameters()
])

# Dilute the plasma sample
n_states = mechanistic_model._n_states
parameters[:n_states] /= 3

# Set thrombomodulin concentration to zero
parameter_names = np.array(mechanistic_model.parameters())
index = np.where(parameter_names == 'central.thrombomodulin_amount')[0]
parameters[index] = 0

# Update coagulation factor (a)XI concentrations
index = np.where(parameter_names == 'central.size')[0]
volume = parameters[index]
index = np.where(
    parameter_names == 'central.coagulation_factor_xi_amount')[0]
initial_xi_conc = parameters[index]
parameters[index] = initial_xi_conc * 0.339
index = np.where(
    parameter_names == 'central.activated_coagulation_factor_xi_amount')[0]
parameters[index] = initial_xi_conc * 0.148

# Set endogenous production rates to zero
index = np.where(parameter_names == 'myokit.input_rate_vk')[0]
parameters[index] = 0
index = np.where(parameter_names == 'myokit.production_rate_fibrinogen')[0]
parameters[index] = 0
index = np.where(
    parameter_names == 'myokit.production_rate_plasminogen')[0]
parameters[index] = 0
index = np.where(
    parameter_names == 'myokit.production_rate_prekrallikrein')[0]
parameters[index] = 0
index = np.where(parameter_names == 'myokit.production_rate_tfpi')[0]
parameters[index] = 0
index = np.where(parameter_names == 'myokit.production_rate_tmod')[0]
parameters[index] = 0
index = np.where(parameter_names == 'myokit.production_rate_v')[0]
parameters[index] = 0
index = np.where(parameter_names == 'myokit.production_rate_viii')[0]
parameters[index] = 0
index = np.where(parameter_names == 'myokit.production_rate_xi')[0]
parameters[index] = 0
index = np.where(parameter_names == 'myokit.production_rate_xii')[0]
parameters[index] = 0
index = np.where(parameter_names == 'myokit.production_rate_xiii')[0]
parameters[index] = 0

# Make sure that only compounds in central compartment are considered
index = np.where(parameter_names == 'dose.drug_amount')[0]
parameters[index] = 0
index = np.where(
    parameter_names == 'peripheral_vitamin_k.vitamin_k_peripheral_amount'
)[0]
parameters[index] = 0
index = np.where(
    parameter_names == 'myokit.transition_rate_vk_central_to_peripheral'
)[0]
parameters[index] = 0
index = np.where(
    parameter_names == 'myokit.transition_rate_vk_peripheral_to_central'
)[0]
parameters[index] = 0

# Change time units to seconds (for numerical stability)
indices = []
for idp, name in enumerate(parameter_names):
    if 'rate' in name:
        indices.append(idp)
parameters[indices] /= 60 * 60

# Add 300 nM contact activator
index = np.where(
    parameter_names == 'central.contact_system_activator_amount')[0]
parameters[index] = 300 * volume

# Integrate fibrin concentration curve
times = np.linspace(start=0, stop=90, num=1000)
delta_time = (times[1] - times[0])
simulation = mechanistic_model.simulate(parameters, times)
fibrin_auc = np.cumsum(simulation[-1] * delta_time)

Visualise results

In [6]:
# Visualise results
# Figure 1: Change of coagulation factors
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=times,
    y=simulation[0] / simulation[0, 0],
    name='Prothrombin'
))
fig.add_trace(go.Scatter(
    x=times,
    y=simulation[1] / simulation[0, 0],
    name='Thrombin'
))
fig.add_trace(go.Scatter(
    x=times,
    y=simulation[2] / simulation[2, 0],
    name='Coag. factor X'
))
fig.add_trace(go.Scatter(
    x=times,
    y=simulation[3] / simulation[2, 0],
    name='Act. coag. factor X'
))
fig.add_trace(go.Scatter(
    x=times,
    y=simulation[4] / simulation[4, 0],
    name='Fibrinogen'
))
fig.add_trace(go.Scatter(
    x=times,
    y=simulation[5] / simulation[4, 0],
    name='Fibrin'
))
fig.update_layout(
    xaxis_title='Time in seconds',
    yaxis_title=r'% of initial inactivated concentration',
    yaxis_range=[0, 1]
)
fig.show()

# Figure 2: Fibrin integral
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=times,
    y=fibrin_auc,
))
fig.update_layout(
    xaxis_title='Time in days',
    yaxis_title='Integral of fibrin in nMs',
    yaxis_type='log',
    yaxis_range=[-1, 6]
)
fig.show()

## Reproduce Figure 3 in Wajima et al (2009)

Treatment response to daily 4 mg warfarin dose.

In [7]:
# Define outputs and dosing regimen
outputs_of_interest = [
    'central_warfarin.warfarin_concentration',
    'central.vitamin_k_concentration',
    'central.vitamin_k_hydroquinone_concentration',
    'central.vitamin_k_epoxide_concentration',
    'central.coagulation_factor_ii_concentration',
    'central.coagulation_factor_vii_concentration',
    'central.coagulation_factor_ix_concentration',
    'central.coagulation_factor_x_concentration']
n_states = mechanistic_model._n_states
plasma_sample_states = mechanistic_model.parameters()[:n_states]
mechanistic_model.set_outputs(outputs_of_interest + plasma_sample_states)
mechanistic_model.set_dosing_regimen(dose=4, start=0, period=24, num=20)

# Simulate treatment response over a period of 25 days
times = np.linspace(start=0, stop=24 * 20, num=1000)
parameters = np.array([
    parameters_df[parameters_df.Parameter == p].Value.values[0]
    for p in mechanistic_model.parameters()
])
simulation = mechanistic_model.simulate(parameters=parameters, times=times)

# Perform coagulation test for each plasma sample
inr_test_model, _ = define_wajima_model(patient=True, inr_test=True)
parameters = np.array([
    parameters_df[parameters_df.Parameter == p].Value.values[0]
    for p in inr_test_model.parameters()
])

prothrombin_time = np.empty(shape=len(times))
for ids, plasma_sample in enumerate(simulation[len(outputs_of_interest):].T):
    prothrombin_time[ids] = inr_test_model.compute_prothrombin_time(
        plasma_sample, parameters)

# Standardise PT to INR
prothrombin_time = prothrombin_time / prothrombin_time[0]

# Convert time unit to days
times /= 24

Visualise results

In [8]:
# Figure 1: Warfarin concentration
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=times,
    y=simulation[0],
))
fig.update_layout(
    xaxis_title='Time in days',
    yaxis_title='Warfarin concentration in mg/L',
    yaxis_range=[0, 1]
)
fig.show()

# Figure 2: Vitamin K related compounds
fig = go.Figure()
parameters = np.array([
    parameters_df[parameters_df.Parameter == p].Value.values[0]
    for p in mechanistic_model.parameters()
])
parameter_names = np.array(mechanistic_model.parameters())
index_amount = np.where(parameter_names == 'central.vitamin_k_amount')[0]
index_v = np.where(parameter_names == 'central.size')[0]
initial_conc = parameters[index_amount] / parameters[index_v]
fig.add_trace(go.Scatter(
    x=times,
    y=simulation[1] / initial_conc * 100,
    name='Vitamin K'
))
index_amount = np.where(
    parameter_names == 'central.vitamin_k_hydroquinone_amount')[0]
initial_conc = parameters[index_amount] / parameters[index_v]
fig.add_trace(go.Scatter(
    x=times,
    y=simulation[2] / initial_conc * 100,
    name='Vitamin K hydroquinone'
))
index_amount = np.where(
    parameter_names == 'central.vitamin_k_epoxide_amount')[0]
initial_conc = parameters[index_amount] / parameters[index_v]
fig.add_trace(go.Scatter(
    x=times,
    y=simulation[3] / initial_conc * 100,
    name='Vitamin K epoxide'
))
fig.update_layout(
    xaxis_title='Time in days',
    yaxis_title=r'Fraction of initial concentration in %',
    yaxis_range=[0, 150]
)
fig.show()

# Figure 3: Vitamin K dependent coagulation factors
fig = go.Figure()
index_amount = np.where(
    parameter_names == 'central.coagulation_factor_ii_amount')[0]
initial_conc = parameters[index_amount] / parameters[index_v]
fig.add_trace(go.Scatter(
    x=times,
    y=simulation[4] / initial_conc * 100,
    name='II'
))
index_amount = np.where(
    parameter_names == 'central.coagulation_factor_vii_amount')[0]
initial_conc = parameters[index_amount] / parameters[index_v]
fig.add_trace(go.Scatter(
    x=times,
    y=simulation[5] / initial_conc * 100,
    name='VII'
))
index_amount = np.where(
    parameter_names == 'central.coagulation_factor_ix_amount')[0]
initial_conc = parameters[index_amount] / parameters[index_v]
fig.add_trace(go.Scatter(
    x=times,
    y=simulation[6] / initial_conc * 100,
    name='IX'
))
index_amount = np.where(
    parameter_names == 'central.coagulation_factor_x_amount')[0]
initial_conc = parameters[index_amount] / parameters[index_v]
fig.add_trace(go.Scatter(
    x=times,
    y=simulation[7] / initial_conc * 100,
    name='X'
))
fig.update_layout(
    xaxis_title='Time in days',
    yaxis_title=r'Fraction of initial concentration in %',
    yaxis_range=[0, 150]
)
fig.show()

# Figure 4: Prothtombin time
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=times,
    y=prothrombin_time,
))
fig.update_layout(
    xaxis_title='Time in days',
    yaxis_title='Prothrombin time in INR',
    yaxis_range=[0, 2.5]
)
fig.show()

## Reproduce Figure 4 in Wajima et al (2009)

In [9]:
# Define outputs and dosing regimen
outputs_of_interest = [
    'central_warfarin.warfarin_concentration',
    'central.vitamin_k_concentration',
    'central.vitamin_k_hydroquinone_concentration',
    'central.vitamin_k_epoxide_concentration',
    'central.coagulation_factor_ii_concentration',
    'central.coagulation_factor_vii_concentration',
    'central.coagulation_factor_ix_concentration',
    'central.coagulation_factor_x_concentration']
n_states = mechanistic_model._n_states
plasma_sample_states = mechanistic_model.parameters()[:n_states]
mechanistic_model.set_outputs(outputs_of_interest + plasma_sample_states)
mechanistic_model.set_dosing_regimen(dose=50, start=0, period=24, num=20)

# Simulate treatment response over a period of 25 days
# 1. 0 - 20 days: Warfarin over dose (50mg)
times_1 = np.linspace(start=0, stop=24 * 20, num=1000)
parameters = np.array([
    parameters_df[parameters_df.Parameter == p].Value.values[0]
    for p in mechanistic_model.parameters()
])
simulation_1 = mechanistic_model.simulate(parameters=parameters, times=times_1)

# 2. 20 - 25 days: Single vitamin K dose (1 mg);
# molar mass of Vitamin K 450.7 ng/nmol
times_2 = np.linspace(start=0, stop=24 * 5, num=1000)
parameters[:n_states] = np.copy(simulation_1[-n_states:, -1])
index = np.where(parameter_names == 'central.vitamin_k_amount')[0]
parameters[index] = parameters[index] + 1E6 / 450.7
mechanistic_model.set_dosing_regimen(dose=0)
simulation_2 = mechanistic_model.simulate(parameters=parameters, times=times_2)
simulation = np.hstack((simulation_1, simulation_2))
times = np.hstack((times_1, times_2 + 24 * 20))

# Perform coagulation test for each plasma sample
prothrombin_time = np.empty(shape=len(times))
parameters = np.array([
    parameters_df[parameters_df.Parameter == p].Value.values[0]
    for p in mechanistic_model.parameters()
])
inr_test_model.set_test_duration(120)
for ids, plasma_sample in enumerate(simulation[len(outputs_of_interest):].T):
    prothrombin_time[ids] = inr_test_model.compute_prothrombin_time(
        plasma_sample, parameters)

# Standardise PT to INR
prothrombin_time = prothrombin_time / prothrombin_time[0]

# Convert time unit to days
times /= 24

Visualise results

In [10]:
# Figure 1: Warfarin concentration
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=times,
    y=simulation[0],
))
fig.update_layout(
    xaxis_title='Time in days',
    yaxis_title='Warfarin concentration in mg/L',
    yaxis_range=[0, 15]
)
fig.show()

# Figure 2: Vitamin K related compounds
fig = go.Figure()
parameters = np.array([
    parameters_df[parameters_df.Parameter == p].Value.values[0]
    for p in mechanistic_model.parameters()
])
parameter_names = np.array(mechanistic_model.parameters())
index_amount = np.where(parameter_names == 'central.vitamin_k_amount')[0]
index_v = np.where(parameter_names == 'central.size')[0]
initial_conc = parameters[index_amount] / parameters[index_v]
fig.add_trace(go.Scatter(
    x=times,
    y=simulation[1] / initial_conc * 100,
    name='Vitamin K'
))
index_amount = np.where(
    parameter_names == 'central.vitamin_k_hydroquinone_amount')[0]
initial_conc = parameters[index_amount] / parameters[index_v]
fig.add_trace(go.Scatter(
    x=times,
    y=simulation[2] / initial_conc * 100,
    name='Vitamin K hydroquinone'
))
index_amount = np.where(
    parameter_names == 'central.vitamin_k_epoxide_amount')[0]
initial_conc = parameters[index_amount] / parameters[index_v]
fig.add_trace(go.Scatter(
    x=times,
    y=simulation[3] / initial_conc * 100,
    name='Vitamin K epoxide'
))
fig.update_layout(
    xaxis_title='Time in days',
    yaxis_title=r'Fraction of initial concentration in %',
    yaxis_range=[0, 500]
)
fig.show()

# Figure 3: Vitamin K dependent coagulation factors
fig = go.Figure()
index_amount = np.where(
    parameter_names == 'central.coagulation_factor_ii_amount')[0]
initial_conc = parameters[index_amount] / parameters[index_v]
fig.add_trace(go.Scatter(
    x=times,
    y=simulation[4] / initial_conc * 100,
    name='II'
))
index_amount = np.where(
    parameter_names == 'central.coagulation_factor_vii_amount')[0]
initial_conc = parameters[index_amount] / parameters[index_v]
fig.add_trace(go.Scatter(
    x=times,
    y=simulation[5] / initial_conc * 100,
    name='VII'
))
index_amount = np.where(
    parameter_names == 'central.coagulation_factor_ix_amount')[0]
initial_conc = parameters[index_amount] / parameters[index_v]
fig.add_trace(go.Scatter(
    x=times,
    y=simulation[6] / initial_conc * 100,
    name='IX'
))
index_amount = np.where(
    parameter_names == 'central.coagulation_factor_x_amount')[0]
initial_conc = parameters[index_amount] / parameters[index_v]
fig.add_trace(go.Scatter(
    x=times,
    y=simulation[7] / initial_conc * 100,
    name='X'
))
fig.update_layout(
    xaxis_title='Time in days',
    yaxis_title=r'Fraction of initial concentration in %',
    yaxis_range=[0, 150]
)
fig.show()

# Figure 4: Prothtombin time
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=times,
    y=prothrombin_time,
))
fig.update_layout(
    xaxis_title='Time in days',
    yaxis_title='Prothrombin time in INR',
    yaxis_range=[0, 9]
)
fig.show()